In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.0 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/LopezGG/Analyzing_StereoSet


Cloning into 'Analyzing_StereoSet'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 6), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!unzip /content/Assets

unzip:  cannot find or open /content/Analyzing_StereoSet, /content/Analyzing_StereoSet.zip or /content/Analyzing_StereoSet.ZIP.


In [ ]:
import json
import torch
from transformers import BertTokenizer
from transformers import BertForNextSentencePrediction,BertForMaskedLM
from torch.nn.functional import softmax
import re
from tqdm import tqdm
import os

In [ ]:
devDataFile = r"/content/Assets/dev.json"


## intersentence Analysis

In [ ]:
def loadDevFile(devDataFile,testType):
    with open (devDataFile, encoding='utf-8') as f:
        devData = json.load(f)
    devProcessed = dict()
    for record in devData['data'][testType]:
        rId = record['id']
        
        devProcessed[rId] = {}
        devProcessed[rId]['context'] = record['context']
        devProcessed[rId]['bias_type'] = record['bias_type']
        sentences = []
        for row in record['sentences']:
            tmp = dict()
            tmp['sentence'] = row['sentence']
            tmp['gold_label'] = row['gold_label']
            tmp['id'] = row['id']
            sentences.append(tmp)
        devProcessed[rId]['sentences'] = sentences
    return devProcessed

In [ ]:
devProcessed = loadDevFile(devDataFile,'intersentence')

In [ ]:
model_name = 'bert-base-uncased'
model = BertForNextSentencePrediction.from_pretrained(model_name)
# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
def obtainProb(tokenizer,model,seq_A,seq_B):
    with torch.no_grad():
        # encode the two sequences. Particularly, make clear that they must be 
        # encoded as "one" input to the model by using 'seq_B' as the 'text_pair'
        encoded = tokenizer.encode_plus(seq_A, text_pair=seq_B, return_tensors='pt')
        # NOTE when you print encoded you will see how the token_type_ids are 0 for all tokens in seq_A and 1 for seq_B, 
        # this way the model knows which token belongs to which sequence
        # a model's output is a tuple, we only need the output tensor containing
        # the relationships which is the first item in the tuple
        seq_relationship_logits = model(**encoded)[0]
        # we still need softmax to convert the logits into probabilities
        # index 0: sequence B is a continuation of sequence A
        # index 1: sequence B is a random sequence
        probs = softmax(seq_relationship_logits, dim=1)
        return probs[0][0].item()

Get the model probability for each of the sentences

In [ ]:
for rid,row in tqdm(devProcessed.items()):
    seq_A= row['context']
    for inner_row in row['sentences']:
        seq_B = inner_row['sentence']
        gold_label= inner_row['gold_label']
        inner_row['model_prob'] = obtainProb(tokenizer,model,seq_A,seq_B) # inplace update

100%|██████████| 2123/2123 [13:33<00:00,  2.61it/s]


We are going to choose for cases where the stereotypes and non- stereo have a huge difference in probability for analysis

In [ ]:
def createSmallerRecord(inner_row):
    new_record = dict()
    new_record['sentence'] = inner_row['sentence']
    new_record['gold_label']=inner_row['gold_label']
    new_record['model_prob']=inner_row['model_prob']
    return new_record

In [ ]:
# this is a function for doing offlien analysis on records. It gives records that have a huge difference between stereo and non-stereo
def CreateRecordsAnalysis(devProcessed,threshold=0.75,numDigits=3):
    interesting_records =[]
    for rid,row in devProcessed.items():
        stereo_score=0
        non_stereo_score =0
        newSentences = []
        for inner_row in row['sentences']:
            gold_label= inner_row['gold_label']
            score = round(inner_row['model_prob'] ,numDigits)

            if gold_label =='stereotype':
                stereo_score=score
                newSentences.append(createSmallerRecord(inner_row))
            elif gold_label=='anti-stereotype':
                non_stereo_score=score
                newSentences.append(createSmallerRecord(inner_row))
        if abs(stereo_score-non_stereo_score) >threshold:
            new_row = dict()
            new_row['context'] = row['context']
            new_row['bias_type'] =row['bias_type']
            new_row['sentences'] = newSentences
            interesting_records.append(new_row)
    return interesting_records

In [ ]:
#writes records to separate files
def writeData(outfile,interesting_records):
    with open(outfile,'w',encoding='utf-8') as f:
        for row in interesting_records:
            json.dump(row,f)
            f.write('\n')
    print("Wrote interesting records to ",outfile)
    return

In [ ]:
# we are picking examples for visual analysis with this code
interesting_records = CreateRecordsAnalysis(devProcessed,threshold=0.75)
print("Number of interesting records ",len(interesting_records))
writeData('analysisIntersentenceInteresting.json',interesting_records)

Number of interesting records  350
Wrote interesting records to  analysisIntersentenceInteresting.json


In [ ]:
# writing down interesting records
with open('analysisIntersentenceInteresting.json','w',encoding='utf-8') as f:
    for row in interesting_records:
        json.dump(row,f)
        f.write('\n')

In [ ]:
# to feed to StereoSet eval code, we need {'id': '107427644575c4712bf105f14475af0e', 'score': 0.058055270463228226} per sentence
def CreateScoresListForPred(devProcessed):
    SentenceScores = []
    for index,row in devProcessed.items():
        for inner_row in row['sentences']:
            record =dict()
            record['id'] = inner_row['id']
            record['score'] = inner_row['model_prob']
            SentenceScores.append(record)
    print("Total Number of sentences added ", len(SentenceScores))
    return SentenceScores


In [ ]:
InterSetenceList = CreateScoresListForPred(devProcessed)

Total Number of sentences added  6369


## Intrasentence

In [ ]:
devProcessed = loadDevFile(devDataFile,'intrasentence')

In [ ]:
masked_LM_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# we need to process the input to add cls & sep token. Also we are replacing keyword BLANK with MASK
def tokenizeText(text):
    text = '[CLS]' + text + '[SEP]'
    text = re.sub(r"\bBLANK\b",'[MASK]',text)
    tokenized_text = tokenizer.tokenize(text)
    return tokenized_text

In [ ]:
#this will generate all the required tensors
def ProcessMaskedInput(text):
    tokenized_text = tokenizeText(text)
    masked_index=0
    # we need masked_index to choose the output
    for index,token in enumerate(tokenized_text):
        if token=='[MASK]':
            masked_index=index
            break
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    return tokens_tensor, segments_tensors ,masked_index  

In [ ]:
# To know the prediction we need to know the position of the masked word
def GetTokenIndex(word):
    target = [word]
    target_index= tokenizer.convert_tokens_to_ids(target)[0]
    return target_index

In [ ]:
# this function runs the model for a text and list of candidates and returns 
def FindScoresForCand(text,cand_text_l):
    # Predict all tokens
    with torch.no_grad():
        tokens_tensor, segments_tensors ,masked_index =ProcessMaskedInput(text)
        predictions = masked_LM_model(tokens_tensor, segments_tensors)
        # Transformers models always output tuples.In our case, the first element is the hidden state of the last layer 
        predictions_for_mask = predictions[0][0, masked_index] 
        probs = softmax(predictions_for_mask,dim=0)
        cand_probs = []
        for row in cand_text_l : 
            cand_text = row['sentence']
            cand_tokenized_text = tokenizeText(cand_text)
            word = cand_tokenized_text[masked_index]
            row['model_prob']=probs[GetTokenIndex(word)].item() # in place replacement
        return 

In [ ]:
# call the model and get the probability
for rid,row in tqdm(devProcessed.items()):
    seq_A= row['context']
    candidates = row['sentences']
    FindScoresForCand(seq_A,candidates)

100%|██████████| 2106/2106 [04:36<00:00,  7.63it/s]


In [ ]:
# this is to find interesting records for analysis
interesting_records = CreateRecordsAnalysis(devProcessed,2.0e-02,6)
print("Number of interesting records ",len(interesting_records))
writeData('analysisIntrasentenceInteresting.json',interesting_records)

Number of interesting records  1
Wrote interesting records to  analysisIntrasentenceInteresting.json


In [ ]:
IntraSetenceList = CreateScoresListForPred(devProcessed)

Total Number of sentences added  6318


Gender

Profession

Race

Religion

# Analysis - IntraSentences:

Gender

Race

Profession

Religion